In [6]:
pwd

'/home/fipulab/notebooks/borina'

In [7]:
ls

Notebook.ipynb


In [9]:
import tensorflow as tf
import numpy as np

In [11]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  8105k      0  0:00:10  0:00:10 --:--:-- 14.9M


In [18]:
!tar -xf aclImdb_v1.tar.gz

In [30]:
ls aclImdb

imdbEr.txt  imdb.vocab  README  test/  train/


In [31]:
ls aclImdb/test

labeledBow.feat  neg/  pos/  urls_neg.txt  urls_pos.txt


In [32]:
ls aclImdb/train

labeledBow.feat  pos/    unsupBow.feat  urls_pos.txt
neg/             unsup/  urls_neg.txt   urls_unsup.txt


In [34]:
cat aclImdb/train/pos/6248_7.txt

Being an Austrian myself this has been a straight knock in my face. Fortunately I don't live nowhere near the place where this movie takes place but unfortunately it portrays everything that the rest of Austria hates about Viennese people (or people close to that region). And it is very easy to read that this is exactly the directors intention: to let your head sink into your hands and say "Oh my god, how can THAT be possible!". No, not with me, the (in my opinion) totally exaggerated uncensored swinger club scene is not necessary, I watch porn, sure, but in this context I was rather disgusted than put in the right context.<br /><br />This movie tells a story about how misled people who suffer from lack of education or bad company try to survive and live in a world of redundancy and boring horizons. A girl who is treated like a whore by her super-jealous boyfriend (and still keeps coming back), a female teacher who discovers her masochism by putting the life of her super-cruel "lover" 

In [45]:
!rm -r aclImdb/train/unsup

rm: cannot remove 'aclImdb/train/unsup': No such file or directory


In [43]:
ls aclImdb

imdbEr.txt  imdb.vocab  README  test/  train/


In [44]:
ls aclImdb/train

labeledBow.feat  neg/  pos/  unsupBow.feat  urls_neg.txt  urls_pos.txt


In [47]:
batch_size = 32

In [55]:
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [56]:
print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")

Number of batches in raw_train_ds: 625


In [57]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [58]:
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")

Number of batches in raw_val_ds: 157


In [59]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 25000 files belonging to 2 classes.


In [60]:
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

Number of batches in raw_test_ds: 782


In [61]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

2021-12-26 12:15:22.568132: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-26 12:15:22.618995: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3900025000 Hz


b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

In [63]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re

In [67]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )

In [68]:
max_features = 20000
embedding_dim = 128
sequence_length = 500

In [69]:
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

In [81]:
text_ds = raw_train_ds.map(lambda x, y: x)

In [82]:
vectorize_layer.adapt(text_ds)

In [75]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [76]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [77]:
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

In [92]:
from tensorflow.keras import layers

In [93]:
inputs = tf.keras.Input(shape=(None,), dtype="int64")

In [94]:
# Layer to map those vocab indices into a space of dimensionality 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

In [95]:
# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

In [97]:
# Vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

In [98]:
# Project onto a single unit output layer, and squash it with a sigmoid
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)
model = tf.keras.Model(inputs, predictions)

In [99]:
# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [100]:
epochs = 3

In [101]:
# Fit the model using the train and test datasets.
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3


2021-12-26 12:53:05.553850: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-12-26 12:53:08.162656: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2021-12-26 12:53:12.890186: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-12-26 12:53:16.398006: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


625/625 [==============================] - 26s 18ms/step - loss: 0.5045 - accuracy: 0.7093 - val_loss: 0.3076 - val_accuracy: 0.8728
Epoch 2/3
625/625 [==============================] - 11s 17ms/step - loss: 0.2275 - accuracy: 0.9112 - val_loss: 0.3060 - val_accuracy: 0.8780
Epoch 3/3
625/625 [==============================] - 11s 17ms/step - loss: 0.1149 - accuracy: 0.9578 - val_loss: 0.3868 - val_accuracy: 0.8768


In [103]:
model.evaluate(test_ds)

782/782 [==============================] - 3s 3ms/step - loss: 0.4087 - accuracy: 0.8638


[0.40872645378112793, 0.8638399839401245]

In [104]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

In [106]:
# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

In [108]:
# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)

782/782 [==============================] - 5s 6ms/step - loss: 0.4087 - accuracy: 0.8638


[0.4087267518043518, 0.8638399839401245]